# arXiv.org数据检索

arXiv.org是一个收录科学文献预印本的在线数据库，目前包含了超过50万篇文章，并且以每个月5000篇的速度增长着。目前，这个数据库包含：物理学，数学，计算机科学，定量生物学，量化金融、统计学、电子工程与系统科学几大分类。其最重要的特点就是“开放式获取”，每个人都可以免费地访问全文数据。如果想要看看某个领域的最新发展，知道大家都在干什么，找到自己感兴趣的研究领域最近发表的文章，使用arXiv.org数据库非常合适。

利用arXiv.org，检索最近一周在某个领域上发表的文章。arXiv.org的领域可以分为上述的8大类，而物理学又可以细分为更多的领域，当然每个领域也可以包含若干个主题。因此，检索方式为根据用户选择的某个领域，获得这个领域最近一周内提交的文章信息，包含文章的题目、作者、主题、注释、期刊参考。

爬取所用的网页为：'https://arxiv.org/'。
根据8个不同的邻域，分别定义8个不同的函数。下面先写出基本的代码框架：

In [ ]:
#默认选择的区间为过去一周中所提交的论文。
#按照arXiv上的分类，给出了8个学科领域，其中每个学科领域又可以细分为多个学科领域。
import requests
from bs4 import BeautifulSoup
import pandas as pd

#获得当前页面的HTML内容
def getHTMLText(url):
    return ''

#获得文章的题目、作者、主题、注释、参考期刊信息
def getPageInfo(url,lst):
    pass

#按照用户输入的检索的页面数，获得所有页码的URL
def getURLList(keyword):
    return ''

#只有物理学下细分了有更多的领域
def searchInPhysics(lst):
    pass
    
def searchInMath(lst):
    pass

def searchInCs(lst):
    pass

def searchInQbio(lst):
    pass

def searchInQfin(lst):
    pass

def searchInStat(lst):
    pass

def searchInEess(lst):
    pass

def searchInEcon(lst):
    pass

def main():
    start_url='https://arxiv.org/'
    html=getHTMLText(start_url)
    soup=BeautifulSoup(html,'html.parser')
    
    classes=[]  #用于存储8种不同领域类别的名称
    
    #爬取8个领域名称
    select=soup.find('select',attrs={'name':'group'})
    for option in select.find_all('option'):
        classes.append(option.text.strip())
    
    #将8种领域类型名称打印出来    
    for i in range(len(classes)):
        print(str(i+1)+'、'+classes[i])
        
    choose=input("请选择检索的领域：")
    info=[]     #用于存储检索得到的论文的信息
    
    #按照用户选择的领域，在所选领域种进行检索
    if choose==str(1):
        searchInPhysics(info)
    elif choose==str(2):
        searchInMath(info)
    elif choose==str(3):
        searchInQbio(info)
    elif choose==str(4):
        searchInCs(info)
    elif choose==str(5):
        searchInQfin(info)
    elif choose==str(6):
        searchInStat(info)
    elif choose==str(7):
        searchInEess(info)
    else:
        searchInEcon(info)

main()

我们以选择计算机科学领域为例，按每一页50条记录显示文章，可以看到网页的URL为'https://arxiv.org/list/cs/pastweek?skip=0&show=50'。
其中cs为计算机科学的英文缩写，每个不同的领域按照英文缩写去标识，如数学为'math'，统计学为'stat'，等等；skip=0表示当前为第1页，skip=50表示第2页，skip=100表示第3页，以此类推；show=50表示每一页显示50条记录。所以searchInMath的函数定义为：

In [ ]:
def searchInCs(lst):
    word='cs'
    for url in getURLList(word):
        getPageInfo(url,lst)

其余7个函数的定义与searchInCs类似，不作一一阐述。而物理学又可以细分为多个领域，因此在searchInPhysics函数中，可以再次获得用户的选择。再次在网页'https://arxiv.org/'
上获得物理学中各个领域的名称。

In [ ]:
def searchInPhysics(lst):
    start_url='https://arxiv.org/'
    html=getHTMLText(start_url)
    soup=BeautifulSoup(html,'html.parser')
    
    physicsClasses=[]   #用于存储所有更细一级的学科领域名称
    abbrs=[]            #用于存储学科领域对应的英文缩写
    word=''             #用于存储每个不同类别对应的英文缩写
    
    #找到所有物理学下的更细一级的学科领域的名称和英文缩写
    for li in soup.find('ul').find_all('li'):
        physicsClasses.append(li.find('a').text)
        abbrs.append(li.find('a').get('href').split('/')[2])
        
    for i in range(len(physicsClasses)):
        print(str(i+1)+'、'+physicsClasses[i])
        
    choose=input('请选择更细一级的检索领域：')
    
    word=abbrs[int(choose)-1]
        
    for url in getURLList(word):
        getPageInfo(url,lst)

用getPageInfo函数获得文章的题目、作者、主题、注释、参考期刊信息。以搜索计算机科学领域为例，URL为'https://arxiv.org/list/cs/pastweek?skip=0&show=50'。
每一条记录的题目、作者、主题、注释、参考期刊信息都可以定位在一个dd标签内，以获得第21条记录的文章的信息为例，其dd标签的内容为：

可以看出，‘题目’为具有属性{'class':'list-title mathjax'}的div标签中的字符串内容；‘作者’为具有属性{'class':'list-authors'}的div标签中的字符串内容；‘主题’为具有属性{'class':'list-subjects'}的div标签中的字符串内容；‘注释’为具有属性{'class':'list-comments mathjax'}的div标签中的字符串内容；‘期刊参考’为具有属性{'class':'list-journal-ref'}的div标签中的字符串内容。注意的是，所有文章都有题目、作者和主题，但是有的文章没有注释或者期刊参考。

getPageInfo函数可以定义为：

In [5]:
def getPageInfo(url,lst):
    html=getHTMLText(url)
    if html=='':
        return
    soup=BeautifulSoup(html,'html.parser')
    
    for dd in soup.find_all('dd'):
        infoDict={}
        infoDict['题目']=dd.find('div',attrs={'class':'list-title mathjax'}).text.split(':')[1].strip()
        infoDict['作者']=dd.find('div',attrs={'class':'list-authors'}).text.split(':')[1].strip().replace(',','').split('\n')
        infoDict['主题']=dd.find('div',attrs={'class':'list-subjects'}).text.split(':')[1].strip().split(';')
        
        if dd.find('div',attrs={'class':'list-comments mathjax'})==None:
            pass
        else:
            infoDict['注释']=dd.find('div',attrs={'class':'list-comments mathjax'}).text.replace('Comments:','').strip()
            
        if dd.find('div',attrs={'class':'list-journal-ref'})==None:
            pass
        else:
            infoDict['期刊参考']=dd.find('div',attrs={'class':'list-journal-ref'}).text.replace('Journal-ref:','').strip()
            
        lst.append(infoDict)

In [ ]:
最后，可以将得到的数据转化为DataFrame的格式，然后存储为一个excel文件，可用于后续的数据分析。下面给出完整的代码实现：

In [ ]:
#默认选择的区间为过去一周中所提交的论文。
#按照arXiv上的分类，给出了8个学科领域，其中每个学科领域又可以细分为多个学科领域。
import requests
from bs4 import BeautifulSoup
import pandas as pd

#获得当前页面的HTML内容
def getHTMLText(url):
    try:
        r=requests.get(url)
        r.raise_for_status()
        r.encoding='utf-8'
        return r.text
    except:
        return ''

#获得文章的题目、作者、主题、注释、参考期刊信息
def getPageInfo(url,lst):
    html=getHTMLText(url)
    if html=='':
        return
    soup=BeautifulSoup(html,'html.parser')
    
    for dd in soup.find_all('dd'):
        infoDict={}
        infoDict['题目']=dd.find('div',attrs={'class':'list-title mathjax'}).text.split(':')[1].strip()
        infoDict['作者']=dd.find('div',attrs={'class':'list-authors'}).text.split(':')[1].strip().replace(',','').split('\n')
        infoDict['主题']=dd.find('div',attrs={'class':'list-subjects'}).text.split(':')[1].strip().split(';')
        
        if dd.find('div',attrs={'class':'list-comments mathjax'})==None:
            pass
        else:
            infoDict['注释']=dd.find('div',attrs={'class':'list-comments mathjax'}).text.replace('Comments:','').strip()
            
        if dd.find('div',attrs={'class':'list-journal-ref'})==None:
            pass
        else:
            infoDict['期刊参考']=dd.find('div',attrs={'class':'list-journal-ref'}).text.replace('Journal-ref:','').strip()
            
        lst.append(infoDict)

#按照用户输入的检索的页面数，获得所有页码的URL
def getURLList(keyword):
    pageNumber=input('请输入检索的总页数（按每一页50条记录）：')
    lst=[]  #用于存储所有页码的URL
    
    #获得当前页码的URL
    for i in range(int(pageNumber)):
        lst.append('https://arxiv.org/list/'+keyword+'/pastweek?skip='+str(i*50)+'&show=50')
    return lst

#只有物理学下细分了有更多的领域
def searchInPhysics(lst):
    start_url='https://arxiv.org/'
    html=getHTMLText(start_url)
    soup=BeautifulSoup(html,'html.parser')
    
    physicsClasses=[]   #用于存储所有更细一级的学科领域名称
    abbrs=[]            #用于存储学科领域对应的英文缩写
    word=''             #用于存储每个不同类别对应的英文缩写
    
    #找到所有物理学下的更细一级的学科领域的名称和英文缩写
    for li in soup.find('ul').find_all('li'):
        physicsClasses.append(li.find('a').text)
        abbrs.append(li.find('a').get('href').split('/')[2])
        
    for i in range(len(physicsClasses)):
        print(str(i+1)+'、'+physicsClasses[i])
        
    choose=input('请选择更细一级的检索领域：')
    
    word=abbrs[int(choose)-1]
        
    for url in getURLList(word):
        getPageInfo(url,lst)
    
def searchInMath(lst):
    word='math'
    for url in getURLList(word):
        getPageInfo(url,lst)
    

def searchInCs(lst):
    word='cs'
    for url in getURLList(word):
        getPageInfo(url,lst)

def searchInQbio(lst):
    word='q-bio'
    for url in getURLList(word):
        getPageInfo(url,lst)

def searchInQfin(lst):
    word='q-fin'
    for url in getURLList(word):
        getPageInfo(url,lst)

def searchInStat(lst):
    word='stat'
    for url in getURLList(word):
        getPageInfo(url,lst)

def searchInEess(lst):
    word='eess'
    for url in getURLList(word):
        getPageInfo(url,lst)

def searchInEcon(lst):
    word='econ'
    for url in getURLList(word):
        getPageInfo(url,lst)

def main():
    start_url='https://arxiv.org/'
    html=getHTMLText(start_url)
    soup=BeautifulSoup(html,'html.parser')
    
    classes=[]  #用于存储8种不同领域类别的名称
    
    #爬取8个领域名称
    select=soup.find('select',attrs={'name':'group'})
    for option in select.find_all('option'):
        classes.append(option.text.strip())
    
    #将8种领域类型名称打印出来    
    for i in range(len(classes)):
        print(str(i+1)+'、'+classes[i])
        
    choose=input("请选择检索的领域：")
    info=[]     #用于存储检索得到的论文的信息
    
    #按照用户选择的领域，在所选领域种进行检索
    if choose==str(1):
        searchInPhysics(info)
    elif choose==str(2):
        searchInMath(info)
    elif choose==str(3):
        searchInQbio(info)
    elif choose==str(4):
        searchInCs(info)
    elif choose==str(5):
        searchInQfin(info)
    elif choose==str(6):
        searchInStat(info)
    elif choose==str(7):
        searchInEess(info)
    else:
        searchInEcon(info)
        
    data=pd.DataFrame(info)
    data.to_excel('E:/arXiv.xlsx')
    
main()